In [1]:
import json
import re
import pandas as pd
from typing import Dict, List, Tuple, Union
from nbconvert.filters import ipython2python

In [2]:
with open("1_Using-Jupyter-Lab.ipynb") as test_notebook_file:
    test_notebook = json.load(test_notebook_file)["cells"]
pd.DataFrame(test_notebook).tail()

,cell_type,metadata,source,execution_count,outputs
27,code,"{'tags': ['raises-exception', 'flake8-noqa-cel...",[import not_a_valid_module],8.0,"[{'ename': 'ModuleNotFoundError', 'evalue': 'N..."
28,code,"{'tags': ['flake8-noqa-cell-E402-F401', 'flake...","[for i in range(2):\n, print(i)]",9.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
29,code,{},"[%%python\n, print(""foo"")]",10.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
30,code,{'tags': ['flake8-noqa-cell']},"[%ls\n, import __hello__]",11.0,"[{'name': 'stdout', 'output_type': 'stream', '..."
31,code,{},[],NaN,[]


## Input Notebook

In [3]:
def ignore_cell(notebook_cell):
    if not notebook_cell["source"]:
        return True
    elif notebook_cell["cell_type"] != "code":
        return True

    
def get_clean_notebook(notebook_path):
    with open(notebook_path) as notebook_file:
        notebook_cells = json.load(notebook_file)["cells"]        
    for index, cell in list(enumerate(notebook_cells))[::-1]:
        if ignore_cell(cell):
            notebook_cells.pop(index)
        else:
            cell_source = list(enumerate(cell["source"]))[::-1]
            for source_index, source_line in cell_source:
                cell["source"][source_index] = ipython2python(source_line)
    return notebook_cells
    
test_notebook = get_clean_notebook("1_Using-Jupyter-Lab.ipynb")
pd.DataFrame(test_notebook)

,cell_type,execution_count,metadata,outputs,source
0,code,1,{'tags': ['nbval-ignore-output']},[{'data': {'application/json': {'cell': {'!': ...,"[get_ipython().run_line_magic('lsmagic', '')\n]"
1,code,2,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...","[{'name': 'stdout', 'output_type': 'stream', '...","[get_ipython().run_cell_magic('bash', '', '')\..."
2,code,3,{'tags': ['nbval-ignore-output']},"[{'name': 'stdout', 'output_type': 'stream', '...",[get_ipython().system('ls -la')\n]
3,code,4,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...",[{'data': {'text/plain': ['\x1b[1;31mDocstring...,"[get_ipython().run_line_magic('pinfo', '%%bash..."
4,code,5,"{'tags': ['nbval-ignore-output', 'flake8-noqa-...",[{'data': {'text/plain': ['\x1b[1;31mDocstring...,"[get_ipython().run_line_magic('pinfo', '%%bash..."
5,code,6,{'tags': ['flake8-noqa-cell']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,"[get_ipython().run_line_magic('pinfo', 'map')\n]"
6,code,7,{'tags': ['flake8-noqa-cell']},[{'data': {'text/plain': ['\x1b[1;31mInit sign...,"[get_ipython().run_line_magic('pinfo', 'map')\n]"
7,code,8,"{'tags': ['raises-exception', 'flake8-noqa-cel...","[{'ename': 'ModuleNotFoundError', 'evalue': 'N...",[import not_a_valid_module\n]
8,code,9,"{'tags': ['flake8-noqa-cell-E402-F401', 'flake...","[{'name': 'stdout', 'output_type': 'stream', '...","[for i in range(2):\n, print(i)\n]"
9,code,10,{},"[{'name': 'stdout', 'output_type': 'stream', '...","[get_ipython().run_cell_magic('python', '', ''..."


In [4]:
def generate_input_name(notebook_path, input_nr):
    return f"{notebook_path}#In[{input_nr}]"
generate_input_name("1_Using-Jupyter-Lab.ipynb", 7)

'1_Using-Jupyter-Lab.ipynb#In[7]'

In [5]:
def strip_newline(souce_line):
    return souce_line.rstrip("\n")
def add_newline(souce_line):
    return f"{souce_line}\n"
striped_source = list(map(strip_newline, test_notebook[1]["source"]))
print("striped_source:\n","".join(striped_source))
reassembled_source = map(add_newline, striped_source)
print("reassembled_source:\n","".join(reassembled_source))

striped_source:
 get_ipython().run_cell_magic('bash', '', '')ls -la
reassembled_source:
 get_ipython().run_cell_magic('bash', '', '')
ls -la



In [6]:
test_notebook[1]["metadata"].get("tags", None)

['nbval-ignore-output', 'flake8-noqa-cell']

## Flake8 tags to use

### cell tags
Apply rules to all lines of the cell

pattern: ``flake8-noqa-cell-<rule1>-<rule2>``
    
example: flake8-noqa-cell-E402-F401

#### ignore tag
Whole cell will be ignored

pattern: ``flake8-noqa-cell``

### line tags
Apply rules to line of the cell, with line_nr 

pattern: ``flake8-noqa-line-<line_nr>-<rule1>-<rule2>``
    
example: flake8-noqa-line-1-E402-F401

#### ignore tag
Line will be ignored

pattern: ``flake8-noqa-line-<line_nr>``

example: flake8-noqa-line-1

### Regex
``^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$|^(?P<ignore_cell>flake8-noqa-cell)$|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$``

testcase:
```
flake8-noqa-cell-E402-F401
flake8-noqa-cell
flake8-noqa-line-1-E402-F401
flake8-noqa-line-1
flake8-noqa-line-foo-E402-F401
```

In [7]:
def extract_flake8_tags(notebook_cell: Dict):
    input_nr = notebook_cell["execution_count"]
    flake8_tags = []
    for tag in notebook_cell["metadata"].get("tags", []):
        if tag.startswith("flake8-noqa"):
            flake8_tags.append(tag)
    return {"input_nr": input_nr, "flake8_tags": flake8_tags}
extract_flake8_tags(test_notebook[0])

{'input_nr': 1, 'flake8_tags': []}

In [8]:
extract_flake8_tags(test_notebook[1])

{'input_nr': 2, 'flake8_tags': ['flake8-noqa-cell']}

In [9]:
flake8_tag_pattern = (r"^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)"
                      r"|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$"
                      r"|^(?P<ignore_cell>flake8-noqa-cell)$"
                      r"|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$")
flake8_tag_regex = re.compile(flake8_tag_pattern)
match1 = re.match(flake8_tag_regex, 'flake8-noqa-cell-E402-F401')
print('matching: "flake8-noqa-cell-E402-F401"')
print("cell_rules", match1.group("cell_rules"))
print("line_nr", match1.group("line_nr"))
print("line_rules", match1.group("line_rules"))
match2 = re.match(flake8_tag_regex, 'flake8-noqa-line-1-E402-F401')
print('matching: "flake8-noqa-line-1-E402-F401"')
print("cell_rules", match2.group("cell_rules"))
print("line_nr", match2.group("line_nr"))
print("line_rules", match2.group("line_rules"))

matching: "flake8-noqa-cell-E402-F401"
cell_rules E402-F401
line_nr None
line_rules None
matching: "flake8-noqa-line-1-E402-F401"
cell_rules None
line_nr 1
line_rules E402-F401


## Tags to rules-dict

In [10]:
import warnings

class InvalidFlake8TagWarning(UserWarning):
    pass

def warn_wrong_tag_pattern(flake8_tag: str):
    warnings.warn("flake8-noqa-line/cell-tags should be of form "
                  "'flake8-noqa-cell-<rule1>-<rule2>'|'flake8-noqa-cell'/"
                  "'flake8-noqa-line-<line_nr>-<rule1>-<rule2>'|'flake8-noqa-line-<rule1>', "
                  f"you used: '{flake8_tag}'", InvalidFlake8TagWarning)

def flake8_tag_to_rules_dict(flake8_tag_pattern: re.Pattern, flake8_tag: str) -> Dict[str, List]:
    match = re.match(flake8_tag_pattern, flake8_tag)
    if match:
        if match.group("cell_rules"):
            cell_rules = match.group("cell_rules")
            cell_rules = cell_rules.split("-")
            return {"cell": cell_rules}
        elif match.group("ignore_cell"):
            return {"cell": ["noqa"]}
        elif match.group("line_nr") and match.group("line_rules") :
            line_nr = str(match.group("line_nr"))
            line_rules = match.group("line_rules")
            line_rules = line_rules.split("-")
            return {line_nr: line_rules}
        elif match.group("ignore_line_nr"):
            line_nr = str(match.group("ignore_line_nr"))
            return {line_nr: ["noqa"]}
    warn_wrong_tag_pattern(flake8_tag)
    return {}
    
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-cell-E402-F401'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-cell'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-1-E402-F401'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-1'))
print(flake8_tag_to_rules_dict(flake8_tag_regex, 'flake8-noqa-line-foo-E402-F401'))

{'cell': ['E402', 'F401']}
{'cell': ['noqa']}
{'1': ['E402', 'F401']}
{'1': ['noqa']}
{}


c:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: InvalidFlake8TagWarning: flake8-noqa-line/cell-tags should be of form 'flake8-noqa-cell-<rule1>-<rule2>'|'flake8-noqa-cell'/'flake8-noqa-line-<line_nr>-<rule1>-<rule2>'|'flake8-noqa-line-<rule1>', you used: 'flake8-noqa-line-foo-E402-F401'
  # Remove the CWD from sys.path while we load stuff.


## Total rules dict

In [11]:
def update_rules_dict(total_rules_dict: Dict[str, List], new_rules_dict: Dict[str, List]) -> Dict[str, List]:
    for key, new_rules in new_rules_dict.items():
        old_rules = total_rules_dict.get(key, [])
        if 'noqa' in old_rules+new_rules:
            total_rules_dict[key] = ['noqa']
        else:
            total_rules_dict[key] = old_rules + new_rules
        

def get_flake8_rules_dict(notebook_cell: Dict) -> Tuple[int, Dict[str, List]]:
    flake8_tags = extract_flake8_tags(notebook_cell)
    flake8_tag_regex = (
        r"^flake8-noqa-(cell-(?P<cell_rules>(\w+\d+-?)+)"
        r"|line-(?P<line_nr>\d+)-(?P<line_rules>(\w+\d+-?)+))$"
        r"|^(?P<ignore_cell>flake8-noqa-cell)$"
        r"|^flake8-noqa-line-(?P<ignore_line_nr>\d+)$"
    )
    flake8_tag_pattern = re.compile(flake8_tag_regex)
    total_rules_dict = {}
    for flake8_tag in flake8_tags["flake8_tags"]:
        new_rules_dict = flake8_tag_to_rules_dict(flake8_tag_regex, flake8_tag)
        update_rules_dict(total_rules_dict, new_rules_dict)
    return flake8_tags["input_nr"], total_rules_dict

        

In [12]:
get_flake8_rules_dict(test_notebook[7])

(8, {'cell': ['E402', 'F401']})

In [13]:
get_flake8_rules_dict(test_notebook[8])

(9, {'cell': ['noqa'], '1': ['E402', 'F401', 'W391']})

In [14]:
from nbconvert.filters import ipython2python
ipython2python("%%bash\necho help\necho help2")

"get_ipython().run_cell_magic('bash', '', 'echo help\\necho help2')\n"

In [15]:
ipython2python("%%bash\necho help\necho help2\n")

"get_ipython().run_cell_magic('bash', '', 'echo help\\necho help2')\n"

In [16]:
def generate_rules_list(line_index: int, rules_dict: Dict[str, List]) -> List:
    line_rules = rules_dict.get(str(line_index), [])
    cell_rules = rules_dict.get("cell", [])
    return line_rules + cell_rules

In [17]:
generate_rules_list(1, {'cell': ['noqa'], '1': ['E402', 'F401', 'W391']})

['E402', 'F401', 'W391', 'noqa']

In [18]:
generate_rules_list(2, {'cell': ['noqa'], '1': ['E402', 'F401', 'W391']})

['noqa']

In [19]:
generate_rules_list(2, {'1': ['E402', 'F401', 'W391']})

[]

## check if noqa already exists
To also get the ``\n`` this needs the compiling flag ``re.DOTALL``

pattern: ``^.+?\s*(?P<has_flake8_noqa>[#]\s*noqa\s*[:](\s*\w+\d+[,]?\s*)+)$``

```python
foo  # noqa: E402, Fasd401
foo  # noqa : E402, Fasd401 
"test  # noqa: E402, Fasd401"
```

In [20]:
FLAKE8_NOQA_INLINE_PATTERN = re.compile(
    r"^.+?\s*[#]\s*noqa\s*[:]"
    r"(?P<flake8_noqa_rules>(\s*\w+\d+[,]?\s*)+)(\n)?$"
    r"|^.+?\s*(?P<has_flake8_noqa_all>[#]\s*noqa\s*[:]?\s*)(\n)?$",
    re.DOTALL,
)

In [21]:
def get_inline_flake8_noqa(code_line: str) -> List:
    match = re.match(FLAKE8_NOQA_INLINE_PATTERN, code_line)
    if match:
        flake8_noqa_rules = match.group("flake8_noqa_rules")
        if flake8_noqa_rules:
            flake8_noqa_rules = flake8_noqa_rules.split(",")
            return list(map(lambda line: line.strip(), flake8_noqa_rules))
        elif match.group("has_flake8_noqa_all"):
            return ["noqa"]
        else:
            return []

In [22]:
print(get_inline_flake8_noqa("foo  # noqa: E402, Fasd401"))
print(get_inline_flake8_noqa("foo  # noqa : E402, Fasd401 \n"))
print(get_inline_flake8_noqa("foo  # noqa"))
print(get_inline_flake8_noqa("foo  # noqa \n"))
print(get_inline_flake8_noqa('"foo  # noqa : E402, Fasd401"'))

['E402', 'Fasd401']
['E402', 'Fasd401']
['noqa']
['noqa']
None


In [23]:
%%timeit
get_inline_flake8_noqa("foo  # noqa: E402, Fasd401")

5.16 µs ± 122 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## update flake8 noqa

In [24]:
FLAKE8_NOQA_INLINE_REPLACE_PATTERN = re.compile(r"^(?P<source_code>.+?)\s*(?P<flake8_noqa>[#]\s*noqa\s*[:]?.*)(\n)?$")

def update_inline_flake8_noqa(source_line: str, rules_list: List) -> str:
    inline_flake8_noqa = get_inline_flake8_noqa(source_line)
    if inline_flake8_noqa:
        rules_list = set(inline_flake8_noqa + rules_list)
        source_line = re.sub(
            FLAKE8_NOQA_INLINE_REPLACE_PATTERN, r"\g<source_code>", source_line
        )
    rules_list = sorted(list(rules_list))
    if "noqa" in rules_list:
        noqa_str = ""
    else:
        noqa_str = ", ".join(rules_list)
    if rules_list:
        return f"{source_line}  # noqa: {noqa_str}"
    else:
        return source_line

In [25]:
print(update_inline_flake8_noqa("foo  # noqa: E402, Fasd401", ['noqa']))
print(update_inline_flake8_noqa("foo  # noqa : E402, Fasd401 \n", ['bar', "E402"]))
print(update_inline_flake8_noqa("foo  # noqa", ['bar', "E402"]))
print(update_inline_flake8_noqa("foo  # noqa \n", ['bar', "E402"]))
print(update_inline_flake8_noqa('"foo  # noqa : E402, Fasd401"', ['bar', "E402"]))
print(update_inline_flake8_noqa('"foo  # noqa : E402, Fasd401"', []))

foo  # noqa: 
foo  # noqa: E402, Fasd401, bar
foo  # noqa: 
foo  # noqa: 
"foo  # noqa : E402, Fasd401"  # noqa: E402, bar
"foo  # noqa : E402, Fasd401"


### Implement flake8 
The formatter formats and object of type `flake8.style_guide.Violation` which has following attributes of interest which might need to be overwritten [`filename`, `line_number`, `physical_line`].


In [31]:
import flake8
from flake8.style_guide import Violation
dir(Violation)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'code',
 'column_number',
 'count',
 'filename',
 'index',
 'is_in',
 'is_inline_ignored',
 'line_number',
 'physical_line',
 'text']